In [1]:
# Importing libraries
import pandas as pd
from pathlib import Path
import requests
import json

# Import the API key
from config import geoapify_key


In [3]:
# Importing data
censorship_data_csv = Path("Resources/Censorship_Data_by_Country.csv")
population_data_csv = Path("Resources/Country_Population_2023.csv")

# Reading data
censorship_data = pd.read_csv(censorship_data_csv)
population_data = pd.read_csv(population_data_csv)

In [4]:
# Inspecting data for Censorship
censorship_data.head()

,Flag,Country,Score,Torrents Restricted?,Torrents Banned or Shut Down?,Pornography Restricted?,Pornography Banned?,Political Media Restricted?,Political Media Heavily Censored?,Social Media Restricted?,Social Media Banned?,VPNs Restricted,VPNs Banned?,Messaging and VoIP App Restrictions
0,:af:,Afghanistan,6,Y,N,Y,Y,Y,Y,Y,N,N,N,N
1,:al:,Albania,2,Y,N,N,N,Y,N,N,N,N,N,N
2,:dz:,Algeria,5,Y,N,Y,N,Y,Y,Y,N,N,N,N
3,:ad:,Andorra,1,Y,N,N,N,N,N,N,N,N,N,N
4,:ao:,Angola,2,Y,N,N,N,Y,N,N,N,N,N,N


In [5]:
# Inspecting data for Population
population_data.head()

,#,Country (or dependency),Population (2023)
0,1,India,"1,428,627,663"
1,2,China,"1,425,671,352"
2,3,United States,"339,996,563"
3,4,Indonesia,"277,534,122"
4,5,Pakistan,"240,485,658"


In [7]:
# Cleaning and Renaming Population Data
population_data = population_data.rename(columns={"Country (or dependency)":"Country"})
population_data = population_data.drop(columns=["#"])
population_data.head()


,Country,Population (2023)
0,India,"1,428,627,663"
1,China,"1,425,671,352"
2,United States,"339,996,563"
3,Indonesia,"277,534,122"
4,Pakistan,"240,485,658"


In [8]:
# Merging Censorship and Population
censor_data = pd.merge(censorship_data, population_data, on="Country", how="left")
censor_data.head()

,Flag,Country,Score,Torrents Restricted?,Torrents Banned or Shut Down?,Pornography Restricted?,Pornography Banned?,Political Media Restricted?,Political Media Heavily Censored?,Social Media Restricted?,Social Media Banned?,VPNs Restricted,VPNs Banned?,Messaging and VoIP App Restrictions,Population (2023)
0,:af:,Afghanistan,6,Y,N,Y,Y,Y,Y,Y,N,N,N,N,"42,239,854"
1,:al:,Albania,2,Y,N,N,N,Y,N,N,N,N,N,N,"2,832,439"
2,:dz:,Algeria,5,Y,N,Y,N,Y,Y,Y,N,N,N,N,"45,606,480"
3,:ad:,Andorra,1,Y,N,N,N,N,N,N,N,N,N,N,"80,088"
4,:ao:,Angola,2,Y,N,N,N,Y,N,N,N,N,N,N,"36,684,202"


In [11]:
# Extract Country into a list
country_list = list(censor_data["Country"])
print(country_list)


['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo-Brazzaville', 'Costa Rica', "Côte D'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana ', 'Greece', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kosovo', 'Kuwait

In [12]:
# Geoapify Base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [13]:
# Prepare a list to hold the results
results = []

# Loop through the country list and get coordinates
for country in country_list:

    # Set up the parameters for the request
    params = {
        'text': country,
        'apiKey': geoapify_key,
        'limit': 1  # We only want the first (most relevant) result
    }
    
    # Make the request to the Geoapify API
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:

        # Parse the response JSON to get the coordinates
        data = response.json()
        if data['features']:

            # Usually, the first feature in the list is the most relevant
            feature = data['features'][0]
            coordinates = feature['geometry']['coordinates']

            # Add the country and its coordinates to the results list
            results.append({
                'Country': country,
                'Latitude': coordinates[1],
                'Longitude': coordinates[0]
            })
        else:
            print(f'No results found for {country}')
    else:
        print(f'Failed to get coordinates for {country}: HTTP {response.status_code}')

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Display the DataFrame
print(df)


         Country   Latitude   Longitude
0    Afghanistan  33.768006   66.238514
1        Albania  41.000028   19.999962
2        Algeria  28.000027    2.999983
3        Andorra  40.975336   -0.444278
4         Angola -11.877577   17.569124
..           ...        ...         ...
170    Venezuela   8.001871  -66.110932
171      Vietnam  15.926666  107.965086
172        Yemen  16.347124   47.891527
173       Zambia -14.518912   27.558988
174     Zimbabwe -18.455496   29.746841

[175 rows x 3 columns]
